In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.4, style='darkgrid')
from lr import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from optimizers import SGD, Adam, IRLS
from sklearn.metrics import balanced_accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

c:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Task 3.2

In [2]:
def run_experiments(data_list, optimizer_classes, optimizer_params, include_interactions=False):
    results = []
    for i, data in enumerate(data_list):
        X, y = data
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        for split_index, (train_index, test_index) in enumerate(skf.split(X, y), start=1):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            for optimizer_class in optimizer_classes:
                optimizer = optimizer_class(**optimizer_params.get(optimizer_class.__name__, {}))
                model = LogisticRegression(optimizer, early_stopping_rounds=5, epochs=500, include_interactions=include_interactions)
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                score = balanced_accuracy_score(y_test, y_pred)
                results.append({'dataset': i+1, 'method': optimizer_class.__name__, 'balanced_accuracy': score, 'losses': model.losses, 'split': split_index})
    results_df = pd.DataFrame(results)
    return results_df

In [3]:
def plot_boxplots(results_df):
    for optimizer in results_df['method'].unique():
        optimizer_data = results_df[results_df['method'] == optimizer]
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=optimizer_data, x='dataset', y='balanced_accuracy', color=sns.color_palette()[0])
        plt.title(f'Boxplot of Balanced Accuracy for {optimizer}')
        plt.xlabel('Dataset')
        plt.ylabel('Balanced Accuracy')
        plt.show()

In [ ]:
with open('data_list.pkl', 'rb') as f:
    data_list = pickle.load(f)

optimizer_classes = [SGD, Adam, IRLS]
optimizer_params = {
    'SGD': {'learning_rate': 0.0001},
    'Adam': {'learning_rate': 0.0001, 'beta1': 0.9, 'beta2': 0.999},
    'IRLS': {'tol': 0.0001}
}

results_df = run_experiments(data_list, optimizer_classes, optimizer_params)

In [ ]:
plot_boxplots(results_df)

## Task 3.3

Without early stopping

In [ ]:
def plot_losses(data_list, optimizer_classes, optimizer_params):
    for optimizer_class in optimizer_classes:
        plt.figure(figsize=(10, 6))
        for i in range(len(data_list)):
            dataset = data_list[i]
            X_train, X_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.2, random_state=101)
            optimizer = optimizer_class(**optimizer_params.get(optimizer_class.__name__, {}))
            model = LogisticRegression(optimizer, epochs=500)
            model.fit(X_train, y_train)
            plt.plot(model.losses, label='Dataset ' + str(i+1))
        plt.title(f'Model Losses with {type(optimizer).__name__} optimizer')
        plt.legend()
        plt.xlabel('Epochs')
        plt.ylabel('Log-likelihood')
        plt.show()

In [ ]:
plot_losses(data_list, optimizer_classes, optimizer_params)

With early stopping

In [ ]:
df = results_df[results_df['split'] == 1]
for method in df['method'].unique():
    method_df = df[df['method'] == method]
    plt.figure(figsize=(15, 6))
    plt.title(f'Log-likelihood per Epoch - {method} optimizer')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    for index, row in method_df.iterrows():
        plt.plot(range(1, len(row['losses']) + 1), row['losses'], label=f'Dataset {row["dataset"]}')
    plt.legend()
    plt.show()

## Task 3.4

In [6]:
def run_baseline(data_list, classifiers):
    results = []
    for i, data in enumerate(data_list):
        X, y = data
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        for split_index, (train_index, test_index) in enumerate(skf.split(X, y), start=1):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            for classifier in classifiers:
                model = classifier()
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                score = balanced_accuracy_score(y_test, y_pred)
                results.append({'dataset': i+1, 'method': type(model).__name__, 'balanced_accuracy': score, 'split': split_index})
    results_df = pd.DataFrame(results)
    return results_df

In [5]:
def plot_all(results_df):
    for dataset in results_df['dataset'].unique():
        df = results_df[results_df['dataset'] == dataset]
        plt.figure(figsize=(15, 6))
        sns.boxplot(data=df, x='method', y='balanced_accuracy', color=sns.color_palette()[0])
        plt.title(f'Boxplot of Balanced Accuracy for Dataset {dataset}')
        plt.xticks(ticks=range(len(df['method'].unique())), labels=['SGD', 'ADAM', 'IRLS', 'LDA', 'QDA', 'Decision Tree', 'Random Forest'])
        plt.xlabel('Method')
        plt.ylabel('Balanced Accuracy')
        plt.show()

In [ ]:
classifiers = [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis, DecisionTreeClassifier, RandomForestClassifier]
baseline_results_df = run_baseline(data_list, classifiers)
final_results_df = pd.concat([results_df, baseline_results_df])

In [ ]:
plot_all(final_results_df)

## Task 3.5

In [ ]:
data_list_interactions = data_list[:3]
results_interactions = run_experiments(data_list_interactions, optimizer_classes, optimizer_params, include_interactions=True)

In [ ]:
small_results_df = results_df[results_df['dataset'].isin([1, 2, 3])]
small_results_df['interactions'] = False
results_interactions['interactions'] = True
final_results_df = pd.concat([small_results_df, results_interactions])

In [ ]:
for dataset in final_results_df['dataset'].unique():
    df = final_results_df[final_results_df['dataset'] == dataset]
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='method', y='balanced_accuracy', hue='interactions')
    plt.title(f'Performance of models with and without interactions for Dataset {dataset}')
    plt.xlabel('Method')
    plt.ylabel('Balanced Accuracy')
    plt.legend(title='Interactions')
    plt.show()